In [1]:
import os
import pandas as pd

In [2]:
def reorder_columns_and_save_df(df, columns_list, output_path, file_name):
    if 'chr' not in df.loc[:, '#Chr']:
        df.loc[:, '#Chr'] = 'chr' + df.loc[:, '#Chr'].astype(str)
    df.loc[
        df.loc[:, '#Chr'].isin(['chr21', 'chr22', 21, 22, '21', '22']),
        columns_list
    ].to_csv(
        os.path.join(output_path, file_name),
        sep='\t',
        index=False,
        header=True
    )

In [3]:
dataset = 'LCL'

In [4]:
count_mtx_path = os.path.join(
    '/home/pushkare/NAS1/vanmierl/CRDs_commonpeaksets_matrices/normalized_matrices',
    'PCs_regressed',
    dataset
)

count_mtx_k4me1 = pd.read_csv(
    os.path.join(
        count_mtx_path,
        '_'.join(
            [
                'H3K4me1',
                dataset,
                'RPKMnorm_regressed_qqnorm_forCRD.bed.gz'
            ]
        )
    ),
    sep='\t'
)

count_mtx_k27ac = pd.read_csv(
    os.path.join(
        count_mtx_path,
        '_'.join(
            [
                'H3K27ac',
                dataset,
                'RPKMnorm_regressed_qqnorm_forCRD.bed.gz'
            ]
        )
    ),
    sep='\t'
)

/tmp/ipykernel_3312731/3595873352.py:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  count_mtx_k4me1 = pd.read_csv(
/tmp/ipykernel_3312731/3595873352.py:21: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  count_mtx_k27ac = pd.read_csv(


In [5]:
common_columns = sorted(set(count_mtx_k4me1.columns[6:]).intersection(set(count_mtx_k27ac.columns[6:])))
peak_info_cols = list(count_mtx_k4me1.columns[:6])

In [6]:
output_path = '/data/pushkare/computational_paper/GITHUB/test_data'

In [7]:
reorder_columns_and_save_df(
    count_mtx_k4me1,
    peak_info_cols + common_columns,
    output_path,
    file_name='H3K4me1_chr21_chr22.bed'
)

In [8]:
reorder_columns_and_save_df(
    count_mtx_k27ac,
    peak_info_cols + common_columns,
    output_path,
    file_name='H3K27ac_chr21_chr22.bed'
)

### Extract genotypes for chr21 and chr22

In [3]:
%%bash
bcftools view \
    /data/pushkare/computational_paper/01.mapping_of_chromatin_modules/PHM/all_marks/delaneau_LCL_FIB_final.vcf.gz \
    --regions chr21,chr22 \
    -o /data/pushkare/computational_paper/GITHUB/test_data/LCL_genotypes_chr21_chr22.vcf

In [4]:
%%bash
bgzip /data/pushkare/computational_paper/GITHUB/test_data/LCL_genotypes_chr21_chr22.vcf
tabix -p vcf /data/pushkare/computational_paper/GITHUB/test_data/LCL_genotypes_chr21_chr22.vcf.gz

In [5]:
%%bash
bcftools query \
    -l /data/pushkare/computational_paper/GITHUB/test_data/LCL_genotypes_chr21_chr22.vcf.gz \
    > /data/pushkare/computational_paper/GITHUB/test_data/LCL_genotypes_chr21_chr22_samples.txt

### Bgzip and index .bed files for count matrices

In [6]:
%%bash
scp \
    /data/pushkare/computational_paper/GITHUB/test_data/H3K27ac_chr21_chr22.bed \
    /data/pushkare/computational_paper/GITHUB/test_data/gzipped_bed/H3K27ac_chr21_chr22.bed

scp \
    /data/pushkare/computational_paper/GITHUB/test_data/H34me1_chr21_chr22.bed \
    /data/pushkare/computational_paper/GITHUB/test_data/gzipped_bed/H3K4me1_chr21_chr22.bed


bgzip /data/pushkare/computational_paper/GITHUB/test_data/gzipped_bed/H3K27ac_chr21_chr22.bed
tabix -p bed /data/pushkare/computational_paper/GITHUB/test_data/gzipped_bed/H3K27ac_chr21_chr22.bed.gz

bgzip /data/pushkare/computational_paper/GITHUB/test_data/gzipped_bed/H3K4me1_chr21_chr22.bed
tabix -p bed /data/pushkare/computational_paper/GITHUB/test_data/gzipped_bed/H3K4me1_chr21_chr22.bed.gz